## TODO

1. find out how to fit the tokenized thing in the model
2. put the code in a python script so that we can run it on the competition day
3. how to read the answer.evaluate

# Insert data and tokenize it

In [1]:
#To run this notebook, you need to have the folder "AI_Course_Final" (downloaded from Kaggle) 
#in the parent folder of the git-repository-folder

In [2]:
%load_ext autoreload
%autoreload 2

In [27]:
import pandas as pd
from keras.preprocessing.text import Tokenizer
import copy
import os
from functools import reduce

In [4]:
current = os.getcwd()
print(current) # displays our current directory
parent = os.path.abspath(os.path.join(current, os.pardir)) 
print(parent) # displays the parent directory
path = parent + "/AI_Course_Final/CBTest/data/cbt_test.txt"
print(path) 
os.path.exists(path)
# Check if the wanted path exist. Must be True!!
test = pd.read_csv(path, sep='\n')

/Users/fridahammarberg/Documents/NTHU/intelligent_computing/project/intelligent_computing_project
/Users/fridahammarberg/Documents/NTHU/intelligent_computing/project
/Users/fridahammarberg/Documents/NTHU/intelligent_computing/project/AI_Course_Final/CBTest/data/cbt_test.txt


In [5]:
path_CN_test = parent + "/AI_Course_Final/CBTest/data/cbtest_CN_test_2500ex.txt"
path_CN_train = parent + "/AI_Course_Final/CBTest/data/cbtest_CN_train.txt"
path_CN_valid = parent + "/AI_Course_Final/CBTest/data/cbtest_CN_valid_2000ex.txt"

path_NE_test = parent + "/AI_Course_Final/CBTest/data/cbtest_NE_test_2500ex.txt"
path_NE_train = parent + "/AI_Course_Final/CBTest/data/cbtest_NE_train.txt"
path_NE_valid = parent + "/AI_Course_Final/CBTest/data/cbtest_NE_valid_2000ex.txt"

path_P_test = parent + "/AI_Course_Final/CBTest/data/cbtest_P_test_2500ex.txt"
path_P_train = parent + "/AI_Course_Final/CBTest/data/cbtest_P_train.txt"
path_P_valid = parent + "/AI_Course_Final/CBTest/data/cbtest_P_valid_2000ex.txt"

path_V_test = parent + "/AI_Course_Final/CBTest/data/cbtest_V_test_2500ex.txt"
path_V_train = parent + "/AI_Course_Final/CBTest/data/cbtest_V_train.txt"
path_V_valid = parent + "/AI_Course_Final/CBTest/data/cbtest_V_valid_2000ex.txt"

In [6]:
pd.set_option('display.max_colwidth', -1)

CN_test = pd.read_csv(path_CN_test, sep='\n', delimiter='\t', header=None, names=['sentence','answer','empty','options'])
CN_train = pd.read_csv(path_CN_train, sep='\n', delimiter='\t', header=None, names=['sentence','answer','empty','options'])
CN_valid = pd.read_csv(path_CN_valid, sep='\n', delimiter='\t', header=None, names=['sentence','answer','empty','options'])

NE_test = pd.read_csv(path_NE_test, sep='\n', delimiter='\t', header=None, names=['sentence','answer','empty','options'])
NE_train = pd.read_csv(path_NE_train, sep='\n', delimiter='\t', header=None, names=['sentence','answer','empty','options'])
NE_valid = pd.read_csv(path_NE_valid, sep='\n', delimiter='\t', header=None, names=['sentence','answer','empty','options'])

P_test = pd.read_csv(path_P_test, sep='\n', delimiter='\t', header=None, names=['sentence','answer','empty','options'])
P_train = pd.read_csv(path_P_train, sep='\n', delimiter='\t', header=None, names=['sentence','answer','empty','options'])
P_valid = pd.read_csv(path_P_test, sep='\n', delimiter='\t', header=None, names=['sentence','answer','empty','options'])

V_test = pd.read_csv(path_V_test, sep='\n', delimiter='\t', header=None, names=['sentence','answer','empty','options'])
V_train = pd.read_csv(path_V_train, sep='\n', delimiter='\t', header=None, names=['sentence','answer','empty','options'])
V_valid = pd.read_csv(path_V_valid, sep='\n', delimiter='\t', header=None, names=['sentence','answer','empty','options'])

In [7]:
test_df = pd.concat([CN_test, NE_test, P_test, V_test])

train_df = pd.concat([CN_train, NE_train, P_train, V_train])

valid_df = pd.concat([CN_valid, NE_valid, P_valid, V_valid])

In [8]:
test_data = test_df[['sentence']].copy()
train_data = train_df[['sentence']].copy()
valid_data = valid_df[['sentence']].copy()

In [9]:
# delete the nbrs in the beginning of each sentence
test_data['sentence'] = test_data['sentence'].map(lambda x: x.lstrip('1234567890'))
train_data['sentence'] = train_data['sentence'].map(lambda x: x.lstrip('1234567890'))
valid_data['sentence'] = valid_data['sentence'].map(lambda x: x.lstrip('1234567890'))

In [10]:
test_answer = test_df[['answer']].copy().dropna(axis=0, how='any').reset_index(drop=True)
train_answer = train_df[['answer']].copy().dropna(axis=0, how='any').reset_index(drop=True)
valid_answer = valid_df[['answer']].copy().dropna(axis=0, how='any').reset_index(drop=True)

In [11]:
test_option = test_df[['options']].copy().dropna(axis=0, how='any').reset_index(drop=True)
train_option = train_df[['options']].copy().dropna(axis=0, how='any').reset_index(drop=True)
valid_option = valid_df[['options']].copy().dropna(axis=0, how='any').reset_index(drop=True)

In [12]:
test_option_list = []
k = 0
for x in test_option.iterrows():
    string_opt = test_option.values[k][0].replace("|", " ")
    test_option_list.append(string_opt)
    k += 1
    
test_option_df = pd.DataFrame(test_option_list, columns=['options'])


train_option_list = []
k = 0
for x in train_option.iterrows():
    string_opt = train_option.values[k][0].replace("|", " ")
    train_option_list.append(string_opt)
    k += 1
    
train_option_df = pd.DataFrame(train_option_list, columns=['options'])


valid_option_list = []
k = 0
for x in valid_option.iterrows():
    string_opt = valid_option.values[k][0].replace("|", " ")
    valid_option_list.append(string_opt)
    k += 1
    
valid_option_df = pd.DataFrame(valid_option_list, columns=['options'])

In [13]:
#print(len(test_data))
#print(len(train_data))
#print(len(valid_data))

#print(len(test_answer))
#print(len(train_answer))
#print(len(valid_answer))

#print(len(test_option_df))
#print(len(train_option_df))
#print(len(valid_option_df))


#***


#print(len(test_data_list))
#print(len(train_data_list))
#print(len(valid_data_list))

#print(len(test_answer_list))
#print(len(train_answer_list))
#print(len(valid_answer_list))

#print(len(test_option_list))
#print(len(train_option_list))
#print(len(valid_option_list))

In [14]:
test_data_list = test_data['sentence'].tolist()
train_data_list = train_data['sentence'].tolist()
valid_data_list = valid_data['sentence'].tolist()

test_answer_list = test_answer['answer'].tolist()
train_answer_list = train_answer['answer'].tolist()
valid_answer_list = valid_answer['answer'].tolist()

In [15]:
n = 10
new_test_data_list = copy.deepcopy(test_data_list[:21*2*n])
new_test_answer_list = copy.deepcopy(test_answer_list[:2*n])
new_test_option_list = copy.deepcopy(test_option_list[:2*n])

In [16]:
new_train_data_list = copy.deepcopy(train_data_list[:21*8*n])
new_train_answer_list = copy.deepcopy(train_answer_list[:8*n])
new_train_option_list = copy.deepcopy(train_option_list[:8*n])

In [113]:
print('train:')
print(len(new_train_data_list)/21)
print(len(new_train_answer_list))
print(len(new_train_option_list))
print('test:')
print(len(new_test_data_list)/21)
print(len(new_test_answer_list ))
print(len(new_test_option_list))

train:
80.0
80
80
test:
20.0
20
20


In [17]:
tokenizer = Tokenizer()

In [19]:
#tokenizer.fit_on_texts(test_data_list)
#tokenizer.fit_on_texts(train_data_list)
#tokenizer.fit_on_texts(valid_data_list)

In [83]:
#new_test_data_list[20+21*5] #use this one to test so the result below is correct!

In [116]:
#new_test_data_list/test_data_list to small_test_data_list

t = 0
small_list = []
small_test_data_list = []
for u in new_test_data_list:
    if t < 21:
        small_list.append(u)
        t += 1
    else:
        small_test_data_list.append(reduce(lambda a, b: a + b, small_list))
        small_list = []
        t = 1

small_test_data_list.append(reduce(lambda a, b: a + b, small_list))
#small_test_data_list[5]

#***********************************

#new_train_data_list/train_data_list to small_train_data_list

t = 0
small_list = []
small_train_data_list = []
for u in new_train_data_list:
    if t < 21:
        small_list.append(u)
        t += 1
    else:
        small_train_data_list.append(reduce(lambda a, b: a + b, small_list))
        small_list = []
        t = 1
small_train_data_list.append(reduce(lambda a, b: a + b, small_list))
#***********************************

In [117]:
print(len(small_test_data_list))
print(len(small_train_data_list))

20
80


In [118]:
#tokenizer.fit_on_texts(new_train_data_list)
tokenizer.fit_on_texts(small_train_data_list)

In [119]:
train_data_matrix = tokenizer.texts_to_matrix(small_train_data_list)
train_answer_matrix = tokenizer.texts_to_matrix(new_train_answer_list)
train_options_matrix = tokenizer.texts_to_matrix(new_train_option_list)

In [120]:
test_data_matrix = tokenizer.texts_to_matrix(small_test_data_list)
test_answer_matrix = tokenizer.texts_to_matrix(new_test_answer_list)
test_options_matrix = tokenizer.texts_to_matrix(new_test_option_list)

In [121]:
test_data_matrix

array([[ 0.,  1.,  1., ...,  0.,  0.,  0.],
       [ 0.,  1.,  1., ...,  0.,  0.,  0.],
       [ 0.,  1.,  1., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  1.,  1., ...,  0.,  0.,  0.],
       [ 0.,  1.,  1., ...,  0.,  0.,  0.],
       [ 0.,  1.,  1., ...,  0.,  0.,  0.]])

In [192]:
type(test_data_matrix[0][0])

numpy.float64

In [56]:
#train_data_matrix = tokenizer.texts_to_matrix(train_data_list)
#train_options_matrix = tokenizer.texts_to_matrix(train_option_list)
#train_answer_matrix = tokenizer.texts_to_matrix(train_answer_list)

In [57]:
#test_data_matrix = tokenizer.texts_to_matrix(test_data_list)
#test_options_matrix = tokenizer.texts_to_matrix(test_option_list)
#test_answer_matrix = tokenizer.texts_to_matrix(test_answer_list)

In [58]:
#valid_data_matrix = tokenizer.texts_to_matrix(valid_data_list)
#valid_options_matrix = tokenizer.texts_to_matrix(valid_option_list)
#valid_answer_matrix = tokenizer.texts_to_matrix(valid_answer_list)

# Use Pickle for store the matrices...

https://pythontips.com/2013/08/02/what-is-pickle-in-python/

# Creating model

In [194]:
from keras.layers import Dense, Merge, Dropout, Permute, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.models import Sequential

# Contants
VOCABULARY_SIZE = 729 #53628 ## estimate from the goldilocks paper
HIDDEN_LAYER_SIZE = 64  ## 10 neurons for 10 options
LSTM_OUTPUT_SIZE=10 #TODO: change LSTM SIZE
BATCH_SIZE = 32
EPOCHS_SIZE = 50

#Initial Embedding matrices A, B for story and options
#Each sentence is taken as an n-dimensional vector of float64, with n=VOCABULARY-SIZE
#memory_encoder encodes lines 1-21 from story, answer encoder
memory_encoder = Sequential()
memory_encoder.add(Embedding(input_dim = VOCABULARY_SIZE, output_dim = HIDDEN_LAYER_SIZE,input_length = 729)) #input length = 21
memory_encoder.add(Dropout(0.3))

option_encoder = Sequential()
option_encoder.add(Embedding(input_dim = VOCABULARY_SIZE, output_dim = HIDDEN_LAYER_SIZE,input_length = 729)) #input length = 1
option_encoder.add(Dropout(0.3))

#Dot product between answer and question encoders, produces internal state "u"
match = Sequential()
match.add(Merge([memory_encoder, option_encoder], mode="dot", dot_axes=[2, 2]))

#Story embedded with matrix C to produce output vectors
output_memory_encoder = Sequential()
output_memory_encoder.add(Embedding(input_dim=VOCABULARY_SIZE,output_dim = 729,input_length = 729)) #input length = 21
output_memory_encoder.add(Dropout(0.3))

# Response vector, obtained by summing output vectors
# Output dim: (None, 10 options, 21)
response_vector = Sequential()
response_vector.add(Merge([match, output_memory_encoder], mode="sum"))
response_vector.add(Permute((2, 1)))

#Final embedding for prediction, weighting matrix W
#combine response and question vectors and do backpropagation
prediction = Sequential()
prediction.add(Merge([response_vector, option_encoder], mode="concat", concat_axis=-1))
#prediction.add(LSTM(LSTM_OUTPUT_SIZE)) #for soft memory attention mechanism
prediction.add(Dropout(0.3))
prediction.add(Dense(10)) #for regularization
prediction.add(Activation("softmax")) #for softmax 

prediction.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

#prediction.fit([Xs_train, Xq_train, Xs_train], Y_train, batch_size=BATCH_SIZE, 
#           nb_epoch=NBR_EPOCHS, validation_data=([Xs_test, Xq_test, Xs_test], Y_test))

prediction.fit([train_data_matrix, train_options_matrix, train_data_matrix], best_train_answer_matrix, batch_size=BATCH_SIZE, 
           nb_epoch=EPOCHS_SIZE, validation_data=([test_data_matrix, test_options_matrix, test_data_matrix], best_test_answer_matrix))

#Notes: dot_axis, concat _axis? Permute? 


#Doubts: binary_crossentropy instead of categorical crossentropy for loss function

/Users/fridahammarberg/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:26: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/Users/fridahammarberg/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:36: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/Users/fridahammarberg/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:42: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/Users/fridahammarberg/anaconda/lib/python3.6/site-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 80 samples, validate on 20 samples
Epoch 1/50
80/80 [==============================] - 10s 125ms/step - loss: 2.2619 - acc: 0.2810 - val_loss: 2.1766 - val_acc: 0.7211
Epoch 2/50
80/80 [==============================] - 4s 50ms/step - loss: 2.1573 - acc: 0.6614 - val_loss: 2.0041 - val_acc: 0.8792
Epoch 3/50
80/80 [==============================] - 4s 48ms/step - loss: 1.9844 - acc: 0.8488 - val_loss: 1.7531 - val_acc: 0.9975
Epoch 4/50
80/80 [==============================] - 4s 51ms/step - loss: 1.7329 - acc: 0.9626 - val_loss: 1.4247 - val_acc: 0.9990
Epoch 5/50
80/80 [==============================] - 4s 50ms/step - loss: 1.4090 - acc: 0.9939 - val_loss: 1.0401 - val_acc: 0.9991
Epoch 6/50
80/80 [==============================] - 4s 51ms/step - loss: 1.0285 - acc: 0.9970 - val_loss: 0.6530 - val_acc: 0.9997
Epoch 7/50
80/80 [==============================] - 4s 46ms/step - loss: 0.6537 - acc: 0.9983 - val_loss: 0.3380 - val_acc: 0.9997
Epoch 8/50
80/80 [===================

In [189]:
print(train_data_matrix.shape)
print(train_options_matrix.shape)
print(train_answer_matrix.shape)

(80, 729)
(80, 729)
(80, 729)


In [202]:
from keras.layers import Dense, Merge, Dropout, Permute, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import RNN
from keras.models import Sequential

# Contants
VOCABULARY_SIZE = 729 #53628 ## estimate from the goldilocks paper
HIDDEN_LAYER_SIZE = 10  ## 10 neurons for 10 options
BATCH_SIZE = 20 #32
EPOCHS_SIZE = 10

#Initial Embedding matrices A, B for story and options
#Each sentence is taken as an n-dimensional vector of float64, with n=VOCABULARY-SIZE
#memory_encoder encodes lines 1-21 from story, answer encoder
memory_encoder = Sequential()
memory_encoder.add(Embedding(input_dim = VOCABULARY_SIZE, output_dim = HIDDEN_LAYER_SIZE,input_length=729)) #21
#memory_encoder.add(RNN(cell, input_dim=HIDDEN_LAYER_SIZE, input_lenght=21, return_sequences=False) )
memory_encoder.add(Dropout(0.3))

answer_encoder = Sequential()
answer_encoder.add(Embedding(input_dim = VOCABULARY_SIZE, output_dim = HIDDEN_LAYER_SIZE,input_length=729)) #1
answer_encoder.add(Dropout(0.3))

#Dot product between answer and question encoders, produces internal state "u"
prediction = Sequential()
prediction.add(Merge([memory_encoder, answer_encoder], mode ='dot'))
prediction.add(Dropout(0.3))
prediction.add(Dense(10)) #for regularization
prediction.add(Activation('softmax')) #for softmax 

#Optimizer, loss function, metrics setting
prediction.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#training
prediction.fit([train_data_matrix, train_options_matrix], best_train_answer_matrix, batch_size=BATCH_SIZE, 
           nb_epoch=EPOCHS_SIZE, validation_data=([test_data_matrix, test_options_matrix], best_test_answer_matrix))

#scores = prediction.evaluate([test_data_matrix, test_options_matrix], best_test_answer_matrix)
#print("%s: %.2f%%" % (prediction.metrics_names[1], scores[1]*100))

/Users/fridahammarberg/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:26: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/Users/fridahammarberg/anaconda/lib/python3.6/site-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 80 samples, validate on 20 samples
Epoch 1/10
80/80 [==============================] - 2s 28ms/step - loss: 2.3032 - acc: 0.1759 - val_loss: 2.2873 - val_acc: 0.8654
Epoch 2/10
80/80 [==============================] - 1s 12ms/step - loss: 2.2808 - acc: 0.7207 - val_loss: 2.2541 - val_acc: 0.8654
Epoch 3/10
80/80 [==============================] - 1s 12ms/step - loss: 2.2455 - acc: 0.7914 - val_loss: 2.1947 - val_acc: 0.8654
Epoch 4/10
80/80 [==============================] - 1s 12ms/step - loss: 2.1831 - acc: 0.8616 - val_loss: 2.1014 - val_acc: 0.9997
Epoch 5/10
80/80 [==============================] - 1s 12ms/step - loss: 2.0852 - acc: 0.9493 - val_loss: 1.9679 - val_acc: 0.9997
Epoch 6/10
80/80 [==============================] - 1s 12ms/step - loss: 1.9507 - acc: 0.9869 - val_loss: 1.7890 - val_acc: 0.9997
Epoch 7/10
80/80 [==============================] - 1s 14ms/step - loss: 1.7689 - acc: 0.9955 - val_loss: 1.5610 - val_acc: 0.9997
Epoch 8/10
80/80 [=====================

In [193]:
best_train_answer_matrix = np.expand_dims(train_answer_matrix, -1)
best_test_answer_matrix = np.expand_dims(test_answer_matrix, -1)

In [ ]:
#print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

In [172]:
import numpy as np
#np.set_printoptions(threshold=30)

splitted_train_data_matrix=np.vsplit(train_data_matrix,80)
splitted_train_options_matrix=np.vsplit(train_options_matrix,80)
splitted_train_answer_matrix=np.vsplit(train_answer_matrix,80)
#print(splitted_train_data_matrix[0])
print(splitted_train_data_matrix[0].shape)
print(splitted_train_options_matrix[0].shape)
print(splitted_train_answer_matrix[0].shape)
print("**")
print(splitted_train_answer_matrix[0].shape[1])


(1, 729)
(1, 729)
(1, 729)
**
729


In [171]:
D3_splitted_train_data_matrix=np.asarray(splitted_train_data_matrix)
print(D3_splitted_train_data_matrix.shape)
D3_splitted_train_options_matrix=np.asarray(splitted_train_options_matrix)
print(D3_splitted_train_options_matrix.shape)
D3_splitted_train_answer_matrix=np.asarray(splitted_train_answer_matrix)
print(D3_splitted_train_answer_matrix.shape)

(80, 1, 729)
(80, 1, 729)
(80, 1, 729)
